In [68]:
import os
from ultralytics import YOLO
import cv2
from typing import Any

In [69]:
INPUT_FOLDER = f"../input_video/"
OUTPUT_FOLDER = f"../output_video/"
MODEL_PATH = f'../input_data/best.pt'

model = YOLO(MODEL_PATH)
files = os.listdir(INPUT_FOLDER)
files.remove('.gitkeep')
print(files)

['workers_1(1).mp4']


In [70]:
print(model.names)

{0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask', 4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone', 7: 'Safety Vest', 8: 'machinery', 9: 'vehicle'}


In [71]:
def draw_box(results: Any, frame:'numpy.ndarray') -> 'numpy.ndarray':
    """
    Функция обозначения на изображении класса и скора
    :param results: результат обнаружения с координатами рамок, классом, уверенностью
    :param frame: кадр изображения
    :return: измененный кадр
    """
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        class_name = model.names[int(class_id)]
        if score > 0.4:
            if class_name in ['Person', 'Hardhat']:
                color = (0, 255, 0) if class_name == 'Person' else (3, 44, 252)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                label = f"{class_name} {score:.2f}"
                cv2.putText(frame, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

In [72]:

def main() -> None:
    """
    Основная функция обработки видео
    """
    if not files:
        raise FileNotFoundError("Поместите хотя бы одно видео в input_video")
    for file_name in files:
        file_path = INPUT_FOLDER + file_name
        output_name = OUTPUT_FOLDER + file_name[:-4] + ".mp4"

        cap = cv2.VideoCapture(file_path)
        size = (int(cap.get(3)), int(cap.get(4)))
        fps = cap.get(5)

        out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), fps, size)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame)[0]
            frame = draw_box(results, frame)

            out.write(frame)
            frame_min = frame.copy()
            cv2.imshow(file_name[:-4] + ' detection', frame_min)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


0: 384x640 4 Hardhats, 4 NO-Masks, 5 Persons, 5 Safety Vests, 3 machinerys, 80.7ms
Speed: 0.0ms preprocess, 80.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Hardhats, 5 NO-Masks, 6 Persons, 5 Safety Vests, 2 machinerys, 133.1ms
Speed: 10.2ms preprocess, 133.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 2 NO-Masks, 5 Persons, 7 Safety Vests, 4 machinerys, 91.2ms
Speed: 5.1ms preprocess, 91.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 NO-Mask, 7 Persons, 6 Safety Vests, 4 machinerys, 80.7ms
Speed: 2.0ms preprocess, 80.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Hardhats, 2 NO-Masks, 5 Persons, 7 Safety Vests, 2 machinerys, 70.5ms
Speed: 2.0ms preprocess, 70.5ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 4 NO-Masks, 6 Persons, 6 Safety Vests, 2 machinerys, 80.8ms
Speed: 0.

### Вот и подошла работа к концу. Я понимаю, что код не является идеальным (далеко не идеальным), но я старался сделать его как можно лучше. Проект многому меня научил в части дообучения модели. Спасибо за внимание!